This is script for beta calculation. 

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

def calculate_beta(stock_returns, market_returns):
    covariance_matrix = np.cov(stock_returns, market_returns)
    beta = covariance_matrix[0, 1] / covariance_matrix[1, 1]
    return beta

def get_stock_data(ticker, start_date):
    stock_data = yf.download(ticker, start=start_date, progress=False)
    stock_data['Returns'] = stock_data['Adj Close'].pct_change()
    return stock_data

def calculate_betas(stock_tickers, reference_ticker, lookback_periods):
    end_date = pd.to_datetime('today')
    results = {}
    
    for stock in stock_tickers:
        results[stock] = {}
        for days in lookback_periods:
            start_date = (end_date - pd.Timedelta(days=days)).strftime('%Y-%m-%d')
            
            stock_data = get_stock_data(stock, start_date)
            reference_data = get_stock_data(reference_ticker, start_date)
            
            merged_data = pd.merge(stock_data['Returns'], reference_data['Returns'], left_index=True, right_index=True, suffixes=('_stock', '_reference')).dropna()
            
            stock_returns = merged_data['Returns_stock']
            reference_returns = merged_data['Returns_reference']
            
            beta = calculate_beta(stock_returns, reference_returns)
            results[stock][f'{days}_days'] = beta
    
    return pd.DataFrame(results)

# Define the stocks and the reference stock
stock_tickers = ['SOL-USD', 'AVAX-USD', 'ETH-USD','BNB-USD']
reference_ticker = 'BTC-USD'
lookback_periods = [30, 60, 90, 180, 365]

# Calculate the betas
betas = calculate_betas(stock_tickers, reference_ticker, lookback_periods)

# Transpose the DataFrame for better readability
betas_transposed = betas.transpose()

# Calculate the standard deviation of each stock's betas and add it as a new column
betas_transposed['St_Dev'] = betas_transposed.std(axis=1)

# Display the results
print("Calculating betas for --> ", reference_ticker)
print(betas_transposed)

In [ ]:
import yfinance as yf
import pandas as pd

# Define the list of stock tickers for Borsa Istanbul
# Note: "QNBTR" is assumed to be QNB Finansbank with ticker "QNBFB.IS"
tickers = [
    "AEFES.IS", "AKBNK.IS", "AKSEN.IS", "ALARK.IS", "ARCLK.IS", "ASELS.IS", "ASTOR.IS", "BIMAS.IS",
    "BRSAN.IS", "CIMSA.IS", "DOAS.IS", "DOHOL.IS", "EKGYO.IS", "ENKAI.IS", "ENJSA.IS", "EREGL.IS",
    "FROTO.IS", "GARAN.IS", "GUBRF.IS", "HALKB.IS", "HEKTS.IS", "ISCTR.IS", "KCHOL.IS", "KONTR.IS",
    "KOZAA.IS", "KOZAL.IS", "KRDMD.IS", "MGROS.IS", "ODAS.IS", "OYAKC.IS", "PETKM.IS", "PGSUS.IS",
    "SAHOL.IS", "SASA.IS", "SISE.IS", "SOKM.IS", "TAVHL.IS", "TCELL.IS", "THYAO.IS", "TKFEN.IS",
    "TOASO.IS", "TSKB.IS", "TTKOM.IS", "TUPRS.IS", "ULKER.IS", "VAKBN.IS", "VESTL.IS", "YKBNK.IS",
    "QNBFB.IS"
]

# Download historical data for the last year
data = yf.download(tickers, period="1y", progress=False)['Adj Close']

# Calculate daily returns
returns = data.pct_change()

# Calculate the correlation matrix
correlation_matrix = returns.corr()

# Display the correlation matrix
print("BIST Stocks Correlation Matrix (1 Year):")
print(correlation_matrix)